In [1]:
import mygene
import numpy as np
import pandas as pd

/Users/soel/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
expr = pd.read_csv("raw/GSE40419_LC-87_RPKM_expression.txt", sep = "\t")

/var/folders/1p/3wbksjf106vcv_9_4w07nk140000gn/T/ipykernel_10876/3223763880.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  expr = pd.read_csv("raw/GSE40419_LC-87_RPKM_expression.txt", sep = "\t")


In [3]:
expr = pd.DataFrame(expr.iloc[:, 6:].to_numpy(), index = expr['gene'])
expr = expr.groupby(level=0).mean()

In [4]:
expr = expr.loc[(expr.sum(axis=1) != 0), :]

In [5]:
mg = mygene.MyGeneInfo()
M = {}
i = 0
for gene in expr.index:
    if i % 1000 == 0:
        print(i)
    i += 1
    result = mg.query(gene, scopes="symbol", fields=["ensembl"], species="human", verbose=False)
    hgnc_name = gene
    for hit in result["hits"]:
        if "ensembl" in hit and "gene" in hit["ensembl"]:
            M[gene] = hit["ensembl"]["gene"]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [6]:
translation = []
for i in range(expr.shape[0]):
    sym = expr.index[i]
    if sym in M.keys():
        translation.append(M[sym])
    else:
        translation.append("None")
expr["ENS"] = translation

In [7]:
exp = pd.DataFrame((expr.iloc[:, 2:(expr.shape[1]-1)]).to_numpy(), index = expr["ENS"])
exp = exp.sort_index()
exp = exp.drop(["None"], axis = 0)

In [9]:
exp.to_csv("expression_GEO.csv")

In [ ]:
a = pd.read_csv("expression_GEO.c